In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import os
import glob

folder_path = '/content/drive/MyDrive/AUP_project/AUPs/'
files = glob.glob(os.path.join(folder_path, '*.txt'))

with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())

detected_encoding = result['encoding']

text_data = []

for file_path in files:
    with open(file_path, 'r', encoding=detected_encoding) as f:
        text = f.read()
        text_data.append(text)

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    word_tokens = word_tokenize(text)
    filtered_text = [lemmatizer.lemmatize(w) for w in word_tokens if w not in stop_words and w.isalpha()]
    return filtered_text

# Preprocess all the documents
text_data = [preprocess_text(doc) for doc in text_data]

In [ ]:
pip install gensim

In [ ]:
from gensim.models import Phrases

# bigram model
bigram = Phrases(text_data, min_count=5, threshold=100)

# trigram model
trigram = Phrases(bigram[text_data], threshold=100)

def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram[bigram[doc]] for doc in texts]

text_data = make_bigrams(text_data)
text_data = make_trigrams(text_data)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Join back the tokenized words
text_data_joined = [' '.join(doc) for doc in text_data]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text_data_joined)


In [ ]:
from sklearn.decomposition import NMF

n_topics = 10
nmf = NMF(n_components=n_topics, random_state=1)
nmf.fit(X)

def get_top_words(model, feature_names, n_top_words):
    topics = []
    for topic_idx, topic in enumerate(model.components_):
        topics.append([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
    return topics

n_top_words = 10
tf_feature_names = vectorizer.get_feature_names_out()
topic_words = get_top_words(nmf, tf_feature_names, n_top_words)


In [ ]:
import nltk

def contains_topic_word(sentence, topic_words):
    words = word_tokenize(sentence.lower())
    return any(word in topic_words for word in words)

# Join the tokenized words back into sentences
text_data_joined = [' '.join(doc) for doc in text_data]

for i, words in enumerate(topic_words):
    topic_sentences = [sentence for document in text_data_joined for sentence in nltk.sent_tokenize(document) if contains_topic_word(sentence, words)]
    print(f"Topic #{i}:")
    print(topic_sentences[:1])


In [ ]:
disallowed_indicator_words = ["not", "prohibit", "ban", "disallow", "forbid"]

def contains_disallowed_indicator_word(sentence):
    words = word_tokenize(sentence.lower())
    return any(word in disallowed_indicator_words for word in words)

disallowed_sentences = [sentence for document in text_data_joined for sentence in nltk.sent_tokenize(document) if contains_disallowed_indicator_word(sentence)]

print(disallowed_sentences[:5])


In [ ]:
# process before training NMF
disallowed_sentences = [sentence for document in text_data_joined for sentence in nltk.sent_tokenize(document) if contains_disallowed_indicator_word(sentence)]

# train the NMF model on these sentences only
vectorizer = TfidfVectorizer()
X_disallowed = vectorizer.fit_transform(disallowed_sentences)

nmf_disallowed = NMF(n_components=n_topics, random_state=1)
nmf_disallowed.fit(X_disallowed)

tf_feature_names_disallowed = vectorizer.get_feature_names_out()
topic_words_disallowed = get_top_words(nmf_disallowed, tf_feature_names_disallowed, n_top_words)

topic_words_disallowed

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 10
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=100,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda.fit(X)

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % (topic_idx+1)
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

n_top_words = 10
tf_feature_names = vectorizer.get_feature_names_out()
print_top_words(lda, tf_feature_names, n_top_words)


In [ ]:
from sklearn.cluster import KMeans

n_clusters = 10
km = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=100, n_init=1)
km.fit(X)

order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

for i in range(n_clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()


In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

pca = PCA(n_components=2).fit(X.toarray())
data2D = pca.transform(X.toarray())

km = KMeans(n_clusters=num_clusters).fit(data2D)
clusters = km.labels_.tolist()

# Plotting the cluster obtained using k means
plt.figure(figsize=(10, 10))
scatter = plt.scatter(data2D[:, 0], data2D[:, 1], c=km.labels_, s=50, cmap='viridis')
plt.scatter(km.cluster_centers_[:, 0], km.cluster_centers_[:, 1], s=300, c='red')
plt.title('Clusters of documents')
plt.show()

In [ ]:
import requests
import pandas as pd
import re

def get_sector_prediction(url):
    prompt = f"Requirement: Please state the sector in the first word of your response. Question: Based on the URL '{url}', what would be the most likely sector for this business?"

    response = requests.post(
        'https://api.openai.com/v1/completions',
        headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer',
        },
        json={
            'model': 'text-davinci-003',
            'prompt': prompt,
            'max_tokens': 100,
            'temperature': 0.5,
        },
    )
    response_json = response.json()

    try:
        # Extract the first word from the response
        first_word = response_json['choices'][0]['text'].split()[0]
        return first_word
    except KeyError:
        print(response_json)  # If there's an error, print the response
        return None

def extract_sector_from_response(response):
    sectors = ['Technology', 'Telecommunications', 'Healthcare', 'Finance', 'Education', 'Government', 'Retail', 'Entertainment']
    detected_sectors = []

    for sector in sectors:
        if sector.lower() in response.lower():
            detected_sectors.append(sector)

    if not detected_sectors:
        detected_sectors.append('Other')

    return detected_sectors[0]


# Load your CSV
df = pd.read_csv('/content/drive/MyDrive/AUP_project/master_219.csv')

sectors = []
# Apply sector prediction for each URL
for url in df['url']:
    sector = get_sector_prediction(url)
    sectors.append(sector)
    print(f'URL: {url}, Predicted Sectors: {sector}')

In [ ]:
pip install transformers

In [ ]:
import requests
import pandas as pd
import os
from nltk.tokenize import sent_tokenize
import chardet

def truncate_text(text, max_length):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)
    truncated_text = ""
    for sentence in sentences:
        if len(truncated_text + sentence) <= max_length:
            truncated_text += sentence
        else:
            break
    return truncated_text

# Set API key

directory = '/content/drive/MyDrive/AUP_project/AUPs/'

with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())
detected_encoding = result['encoding']

# Loop over all text files in the directory
for filename in os.listdir(directory):
    if filename in df['TextFile'].values:
        continue
    if filename.endswith(".txt"):
        with open(os.path.join(directory, filename), 'r', encoding=detected_encoding, errors='ignore') as f:
            text = f.read()

        # Truncate the text to fit within the model's token limit (approximately 2048 tokens)
        truncated_text = truncate_text(text, 2048)

        # Define the prompt
                # Define the prompt
        prompt = f"please identify every action that the website disallows users to do in the following acceptable user policy and give that action a category label (one to three words that briefly summarize the disallowed action). Please format your answer strictly in a CSV format where the two columns are: 'Disallowed Actions' and 'Label'.\n\n{truncated_text}"

        # Send the request to the GPT-3 model
        response = requests.post(
            'https://api.openai.com/v1/completions',
            headers={
                'Content-Type': 'application/json',
                'Authorization': f'Bearer {api_key}',
            },
            json={
                'model': 'text-davinci-003',
                'prompt': prompt,
                'max_tokens': 2048,
                'temperature': 0.5,
            },
        )

        # Parse the response
        response_data = response.json()['choices'][0]['text'].strip().split('\n')

        # Identify the start of the disallowed actions and labels
        start_index = response_data.index('Disallowed Actions,Label') + 1

        # Parse the disallowed actions and labels
        for row in response_data[start_index:]:
            if ',' in row:  # Only process rows that contain a comma
                disallowed_action, label = row.split(',', 1)  # Split on the first comma only
                df = df.append({"TextFile": filename, "Disallowed Action": disallowed_action.strip(), "Label": label.strip()}, ignore_index=True)
# Save the dataframe to a CSV file
df.to_csv('output.csv', index=False)

In [ ]:
df.to_csv('/content/drive/MyDrive/AUP_project/Raw_Disallows.csv', index=False)

In [ ]:
df

In [ ]:
import numpy as np

np.unique(np.array(df.Label))

In [ ]:
np.unique(np.array(df.TextFile))

In [ ]:
import nltk
nltk.download('punkt')


In [ ]:
label_to_category = {
    # Abuse and Harassment
    'Abuse': 'Abuse and Harassment',
    'Abuse of Resources': 'Abuse and Harassment',
    'Abusive': 'Abuse and Harassment',
    'Abusive Behavior': 'Abuse and Harassment',
    'Abusive Content': 'Abuse and Harassment',
    'Bullying': 'Abuse and Harassment',
    'Harassing': 'Abuse and Harassment',
    'Harassing Content': 'Abuse and Harassment',
    'Harassment': 'Abuse and Harassment',
    'Harassment/Bullying': 'Abuse and Harassment',
    'Harassment/Intimidation': 'Abuse and Harassment',
    'Hate': 'Abuse and Harassment',
    'Hate Speech': 'Abuse and Harassment',
    'Hateful': 'Abuse and Harassment',
    'Hateful content': 'Abuse and Harassment',
    # And so on...

    # Illegal Activities
    'Illegal': 'Illegal Activities',
    'Illegal Actions': 'Illegal Activities',
    'Illegal Activities': 'Illegal Activities',
    'Illegal Activity': 'Illegal Activities',
    'Illegal Content': 'Illegal Activities',
    'Illegal Domain Names': 'Illegal Activities',
    'Illegal Use': 'Illegal Activities',
    'Illegal activities': 'Illegal Activities',
    'Illegal activity': 'Illegal Activities',
    'Illegal behavior': 'Illegal Activities',
    'Illegal content': 'Illegal Activities',
    'Illegal material': 'Illegal Activities',
    'Illegal/Fraudulent': 'Illegal Activities',
    'Illegal/Fraud': 'Illegal Activities',
    'Law violation': 'Illegal Activities',
    'Law-breaking': 'Illegal Activities',
    'Lawbreaking': 'Illegal Activities',

    # Content Issues
    'Adult Content': 'Content Issues',
    'Adult Services': 'Content Issues',
    'Content Removal': 'Content Issues',
    'Content Standards': 'Content Issues',
    # Continue with other labels for this category...

    # Fraudulent Activities
    'Fraud': 'Fraudulent Activities',
    'Fraudulent': 'Fraudulent Activities',
    'Fraudulent Activities': 'Fraudulent Activities',
    'Fraudulent Interaction': 'Fraudulent Activities',
    # Continue with other labels for this category...

    # add other categories
}

df['New_Label'] = df['Label'].map(label_to_category)

# Check for any labels that were not included in the label_to_category dictionary
# assign a new_label value of NaN
missing_labels = df['Label'][df['New_Label'].isnull()].unique()

print('Labels not included in label_to_category:')
for label in missing_labels:
    print(label)